<a href="https://colab.research.google.com/github/Fernigithub/Agreed/blob/main/Agreed_filter_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Run this cell to start Installing and Authenticate
# Installs geemap package
import subprocess

try:
        import geemap 
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
        print('geopandas package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geopandas'])
        print('tsmoothie package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'tsmoothie'])
        print('plotly.express package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'plotly.express'])
        subprocess.check_call(["python", '-m', 'pip', 'install', 'tornado==5.1.0'])
        

# Authenticates and initializes Earth Engine
import ee

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

import warnings
warnings.filterwarnings("ignore")
import ipywidgets as widgets
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from tsmoothie.smoother import LowessSmoother
import plotly.express as px
import plotly.graph_objects as go
from datetime import date
import ee
import geemap
import requests

try:
        from google.colab import files
        from google.colab import drive
        IN_COLAB = True
except:
        IN_COLAB = False



geemap package not installed. Installing ...


In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Run this cell to select GeoJSON file

user = "Fernigithub"
repo = "files"
gdf = None
url = f"https://api.github.com/repos/{user}/{repo}/git/trees/main?recursive=1"
r = requests.get(url)
res = r.json()

file_list=[]
for file in res["tree"]:
    file_list.append(file["path"])
    
checkitems = [widgets.Checkbox(value=False, description=label.split('.')[0]) for label in file_list]

def sel_f (b):
    global gdf
    selected_data = []
    for j,item in enumerate(checkitems):
        if item.value == True:
            selected_data.append(item.description)

    links = [str(f'https://raw.githubusercontent.com/Fernigithub/files/main/{x}.geojson') for x in selected_data]

    gdf = gpd.GeoDataFrame()
    for item in links:
      gdf = gdf.append(gpd.read_file(item))

for item in checkitems:
    item.observe(sel_f,names='value')
    
widgets.GridBox(checkitems, layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))

GridBox(children=(Checkbox(value=False, description='Frog_selsey_outwoods'), Checkbox(value=False, description…

In [ ]:
#@title Run this cell to start or Reset the selection
# Functions

def smooth(serie,smooth_frac=0.02,itera=3):
    serieResult = pd.Series(serie.values)
    serieindex = pd.Series(serie.index)
    smoother = LowessSmoother(smooth_fraction=smooth_frac, iterations=itera)
    dataSmooth = smoother.smooth(serieResult.values)
    return pd.Series(dataSmooth.smooth_data[0],index=serieindex)
    
def hampel_filter_pandas(input_series, window_size, n_sigmas=1):
    k = 1.4826 
    new_series = input_series.copy()
    MAD = lambda x: np.median(np.abs(x - np.median(x)))
    rolling_median = input_series.rolling(window=2*window_size, center=True).mean()
    rolling_mad = k * input_series.rolling(window=2*window_size, center=True).apply(MAD)
    diff = (input_series - rolling_median) * -1 
    indices = list(np.argwhere(diff.values > (n_sigmas * rolling_mad.values)).flatten())
    new_series[indices] = rolling_median[indices]
    return new_series

def hampel_filter_pandas_positivas(input_series, window_size, n_sigmas=1):
    k = 1.4826 
    new_series = input_series.copy()
    MAD = lambda x: np.median(np.abs(x - np.median(x)))
    rolling_median = input_series.rolling(window=2*window_size, win_type = 'gaussian', center=True).mean(std=1)
    rolling_mad = k * input_series.rolling(window=2*window_size, center=True).apply(MAD)
    diff = (input_series - rolling_median)
    indices = list(np.argwhere(diff.values > (n_sigmas * rolling_mad.values)).flatten())
    new_series[indices] = rolling_median[indices]
    return new_series

def cloud (image): 
    image = ee.Image(image)
    clouds = ee.Image(ee.List(image.get('cloud_mask'))).select('probability')
    clean = clouds.lte(80)
    qa = image.select('QA60')
    scl = image.select('SCL').eq(4).Or(image.select('SCL').eq(5))
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    finalmask = mask.And(clean)
    return image.updateMask(finalmask)

imageCollection = ee.ImageCollection("COPERNICUS/S2")
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask') \
                                    .apply(**{'primary': imageCollection,'secondary': s2Clouds, \
                                    'condition': ee.Filter.equals(**{'leftField': 'system:index', 'rightField': 'system:index'})})



# Aggregation
def aggreg (image_collection, startdate : str ,enddate : str ,timeunit : str ,aggregation : int )-> ee.ImageCollection:
    """ 
    image_collection: ee.ImageCollection
    startdate: string
    enddate: string
    timeunit: string
    aggregation: int
    """
    start = ee.Date(startdate)
    end = ee.Date(enddate)
    timedelta = end.difference(start, timeunit)
    def step (count):
        return start.advance(count, timeunit)
    def filter (date):
        images_g = ee.ImageCollection(image_collection.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.mean()).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))
    img_list = ee.List.sequence(0, timedelta.int(),aggregation) \
        .map(step) \
        .map(filter)

    # Check missing values
    def check (image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image
    return ee.ImageCollection(img_list).map(check, True)

# Aggregation
def aggreg_sum (image_collection, startdate : str ,enddate : str ,timeunit : str, aggregation : int )-> ee.ImageCollection:
    """ 
    image_collection: ee.ImageCollection
    startdate: string
    enddate: string
    timeunit: string
    aggregation: int
    """
    start = ee.Date(startdate)
    end = ee.Date(enddate)
    timedelta = end.difference(start, timeunit)
    def step (count):
        return start.advance(count, timeunit)
    def filter (date):
        images_g = ee.ImageCollection(image_collection.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.sum()).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))
    imag_list = ee.List.sequence(0, timedelta.int(),aggregation) \
        .map(step) \
        .map(filter)

    # Check missing values
    def check (image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image
    return ee.ImageCollection(imag_list).map(check, True)

def SM_data (table):
    images = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture') \
    .filterDate('2015-06-01',str(date.today())) \
    .filterBounds(table) \
    .select(['susm'])

    start = ee.Date('2015-06-01')
    end = ee.Date('2022-06-30')
    timeunit = 'month'
    timedelta = end.difference(start, timeunit)
    aggregation = 1
    def step (count):
        return start.advance(count, timeunit)

    def filter (date):
        images_g = ee.ImageCollection(images.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.mean()).multiply(1).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))

    img_list = ee.List.sequence(0, timedelta.int(),aggregation).map(step).map(filter)

    # Check missing values
    def check(image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image

    return ee.ImageCollection(img_list).map(check, True)

def GPM_data (table):
    images = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
    .filterDate('2022-01-01',str(date.today())) \
    .filterBounds(table) \
    .select(['precipitationCal'])

    start = ee.Date('2015-06-01')
    end = ee.Date('2022-06-30')
    timeunit = 'month'
    timedelta = end.difference(start, timeunit)
    aggregation = 1
    def step (count):
        return start.advance(count, timeunit)
    def filter (date):
        images_g = ee.ImageCollection(images.filterDate(date,ee.Date(date).advance(aggregation,timeunit)))
        return ee.Image(images_g.sum()).set('system:time_start',ee.Date(date)).set('system:date',ee.Date(date).format('dd-MM-yyyy'))
    img_list = ee.List.sequence(0, timedelta.int(),aggregation) \
        .map(step) \
        .map(filter)

    # Check missing values
    def check (image):
        image = ee.Image(ee.Algorithms.If(ee.Image(image).bandNames(), ee.Image(image)))
        return image
    return ee.ImageCollection(img_list).map(check, True)

def kendall_calc(collection):
  afterFilter = ee.Filter.lessThan(**{'leftField': 'system:time_start','rightField': 'system:time_start'})
  joined = ee.ImageCollection(ee.Join.saveAll('after').apply(**{'primary': collection,'secondary': collection,'condition': afterFilter}))

  def sign(i, j):
    return ee.Image(j).neq(i) \
            .multiply(ee.Image(j).subtract(i).clamp(-1, 1)).int()

  def func_oca(current):
    afterCollection = ee.ImageCollection.fromImages(current.get('after'))
    return afterCollection.map(lambda x : ee.Image(sign(current, x)).unmask(0))

  return ee.ImageCollection(joined.map(func_oca).flatten()).reduce('sum', 2)

def selectall(b):
  farm = b['owner'].description
  data = gdf.loc[gdf['Farm'] == farm].Field.values
  if b.old ==False:
    for check_group in checkbox_list:
      for item in check_group:
        if item.description in data:
          item.value = True
  else:
    for check_group in checkbox_list:
      for item in check_group:
        if item.description in data:
          item.value = False

if gdf is not None:
    gdf.crs = "epsg:4326"
    table_json = gdf.__geo_interface__
    table = ee.FeatureCollection(table_json['features'])
    coll = SM_data(table)
    # kendall = kendall_calc(coll).resample('bicubic').reproject('EPSG:4326',None,5000)
    kendall = kendall_calc(coll)
    # minmax = kendall.reduceRegion(ee.Reducer.minMax(),table ,1000).getInfo()
    # print(minmax)
    GPM_coll = GPM_data(coll)
    palette = ['red', 'white', 'green']

    selected_fields = ee.FeatureCollection([])
    Map = geemap.Map()
    Map.add_basemap('SATELLITE')
    Map.addLayer(table, {}, 'Farm')
    Map.centerObject(table, 12)
    Map.add_labels(table,'Field',font_color='yellow',font_size='10',draggable=False)
    display(Map)

Map(center=[52.61003816061699, -0.8290340885779912], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
#@title Select Time Ranges and values

#Load UK boundaries from FAO dataset
uk = ee.FeatureCollection("FAO/GAUL/2015/level0").filterMetadata('ADM0_CODE','equals',256)
S2_imgColl = ee.ImageCollection("COPERNICUS/S2_SR")
#Cloud masking for Sentinel-2 lvl2A imagery
def cloudmask(image):
    mask = image.select('SCL').gte(8).And(image.select('SCL').neq(11))
    return image.mask(mask.neq(1))

def ndmi(image):
    ndmi = image.normalizedDifference(['B8','B3']).rename('NDMI')
    return image.addBands(ndmi)

S2 = S2_imgColl.map(cloudmask).map(ndmi)


time_start = '2017-01-01' #@param {type:"date"}
time_end =  '2022-07-31' #@param {type:"date"}
NDMI_low = 'day' #@param ["day", "month", "year"]
NDMI_high = 1 #@param {type:"number"}
print('Selected Range is from ' + time_start + ' to ' + time_end)
print('Aggregation ' + str(aggregation) +' '+ aggregation_unit)

